<a href="https://colab.research.google.com/github/cs-amy/project-codebase/blob/main/notebooks/colab_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MSc Project Model Training on Google Colab**
`Author: Ashraf Muhammed Yusuf (23011173)`

This notebook sets up the environment for training a letter classification model on Google Colab.

## 1. Clone the GitHub Repository

First, clone the project's GitHub repository.

In [ ]:
# Delete the project-codebase directory if it exists
!rm -rf project-codebase

!git clone https://github.com/cs-amy/project-codebase.git
%cd project-codebase

## 2. Mount Google Drive (for data files)

If the project's data files are stored in Google Drive, mount it here.

Note: If you do not already have the project data saved to your drive:


1.   Access it here: https://drive.google.com/drive/folders/1sfNG1PkmTPBe1wOSQXZmfdkvR97Hn9lk?usp=sharing
2.   Copy the entire folder to your drive



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("Drive mounted successfully!")

In [ ]:
# Create symbolic links to the data directory
!ln -s /content/drive/MyDrive/MScProject/data data

print("Symbolic links created successfully!")

## 3. Install Dependencies

Install the required packages from the requirements.txt file.

In [ ]:
!pip install -r requirements.txt

# Specific versions of PyTorch with CUDA support
!pip install torch==2.2.2 torchvision==0.17.2 torchaudio==2.2.2 --index-url https://download.pytorch.org/whl/cu118

print("All dependencies installed successfully!")

## 4. Set Up Python Path

Set up the python path correctly.

In [ ]:
import sys
sys.path.append('/content/project-codebase')

print("Python path set up successfully!")

## 5. Model Configuration
Model configuration for the letter classifier.

In [ ]:
config = {
    "model": {
        "architecture": "LetterClassifierCNN",
        "input_shape": [28, 28, 1],  # Height, Width, Channels
        "num_classes": 26,  # a-z
        "dropout_rate": 0.5 # Dropout rate for the model
    },
    "training": {
        "epochs": 100,
        "batch_size": 64,  # Will be automatically adjusted based on available memory
        "learning_rate": 0.01, # Learning rate for the optimizer
        "weight_decay": 0.0001, # Weight decay for the optimizer
        "optimizer": "adam", # Optimizer to use
        "lr_scheduler": {
            "use": True, # Whether to use a learning rate scheduler
            "type": "reduce_on_plateau", # Type of learning rate scheduler
            "patience": 5, # Number of epochs to wait before reducing learning rate
            "factor": 0.5, # Factor by which the learning rate will be reduced
            "min_lr": 0.00001 # Minimum learning rate
        },
        "early_stopping": {
            "use": True, # Whether to use early stopping
            "patience": 15, # Number of epochs to wait before stopping if no improvement
            "min_delta": 0.001 # Minimum change in loss to qualify as an improvement
        }
    },
    "data": {
        "regular": {
            "train_dir": "data/characters/regular/train",
            "test_dir": "data/characters/regular/test"
        },
        "obfuscated": {
            "train_dir": "data/characters/obfuscated/train",
            "test_dir": "data/characters/obfuscated/test"
        },
        "image_size": [28, 28],  # Input image size (height, width)
        "validation_split": 0.2,  # Portion of training data to use for validation
        "shuffle": True, # Whether to shuffle the data
        "augmentation": {
            "use": True, # Whether to apply augmentation to the image
            "rotation_range": 10, # Randomly rotate the image
            "zoom_range": 0.1, # Randomly zoom in or out on the image
            "width_shift_range": 0.1, # Randomly shift the image horizontally
            "height_shift_range": 0.1, # Randomly shift the image vertically
            "brightness_range": [0.8, 1.2], # Randomly adjust brightness of the image
            "random_noise": 0.01 # Add random noise to the image
        }
    },
    "output": {
        "dir": "outputs/letter_classifier",  # Relative to project root
        "save_frequency": 5,  # Save checkpoint every N epochs
        "keep_best": True   # Keep best model based on validation loss
    }
}

print("Model configuration set up successfully!")

## 6. Train Model

Set up and run the training routine.

In [ ]:
from src.train.train import train

train(config)

## 7. Save Results to Google Drive (optional)

Save the trained model and results to Google Drive for persistence.

In [ ]:
# Create directory for results if it doesn't exist
!mkdir -p /content/drive/MyDrive/MScProject/results

# Copy results to Google Drive
!cp -r results/* /content/drive/MyDrive/MScProject/results